In [1]:
pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 991.0 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.8 MB 770.8 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.8 MB 939.4 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.8 MB 804.6 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/1.8 MB 892.5 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.8 MB 1.1 MB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.8 MB 1.2 MB/s eta 0:00:02
   ----------- ---------------------------- 0.5/1.8 MB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 1.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.8 MB 1.5 MB/s eta 0:00:01
   ----------------- ---------------------- 0.8/1.8 MB 1.6 MB/s eta 0:00:01
   -------------

In [6]:
C:\Users\amysu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

SyntaxError: unexpected character after line continuation character (3904657883.py, line 1)

In [2]:
pip install tensorflow

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tensorflow-2.17.1-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.17.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow-2.17.1-cp311-cp311-win_amd64.whl (7.5 kB)
   ---------------------------------------- 0.0/382.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/382.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/382.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/382.3 MB 929.6 kB/s eta 0:06:51
   ---------------------------------------- 0.5/382.3 MB 929.6 kB/s eta 0:06:51
   ---------------------------------------- 0.8/382.3 MB 838.9 kB/s eta 0:07:35
   ---------------------------------------- 1.0/382.3 MB 867.1 kB/s eta 0:07:20
   --------------

In [3]:
pip show tensorflow

Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\amysu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger

In [2]:
# 닉네임 데이터 준비
nicknames = [
    "멋쟁이사자", "푸른하늘", "달빛요정", "별똥별", "밤하늘여행자",
    "고요한밤", "빛나는별", "은하수", "꿈꾸는고양이", "따스한햇살",
    "꽃잎", "바람", "구름", "비", "눈",
    "웃음", "행복", "사랑", "희망", "믿음",
    # ... 더 많은 닉네임 데이터 추가 ...
]

In [3]:

# 데이터 전처리
tokenizer = Tokenizer(char_level=True)  # 문자 단위 토큰화
tokenizer.fit_on_texts(nicknames)
sequences = tokenizer.texts_to_sequences(nicknames)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

In [4]:
# 데이터셋 분할 (80% 훈련, 20% 검증)
split_index = int(len(padded_sequences) * 0.8)
train_sequences = padded_sequences[:split_index]
validation_sequences = padded_sequences[split_index:]

In [5]:
# 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices(train_sequences)
dataset = dataset.map(lambda seq: (seq[:-1], seq[1:]))
dataset = dataset.shuffle(1000).cache().batch(64)

In [6]:
# 검증 데이터셋 생성
validation_dataset = tf.data.Dataset.from_tensor_slices(validation_sequences)
validation_dataset = validation_dataset.map(lambda seq: (seq[:-1], seq[1:]))
validation_dataset = validation_dataset.batch(64).cache()

In [7]:
# 콜백 함수 정의
checkpoint = ModelCheckpoint('model.keras', save_best_only=True)
early_stopping = EarlyStopping(patience=3)
tensorboard = TensorBoard(log_dir='./logs')
csv_logger = CSVLogger('training.log')

In [8]:
# 모델 생성
vocab_size = len(tokenizer.word_index) + 1
model = Sequential()
model.add(Embedding(vocab_size, 64))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(vocab_size, activation='softmax'))  # 또는 activation='sigmoid'

In [9]:
# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')  # 적절한 손실 함수 사용

In [10]:
# 모델 학습
steps_per_epoch = len(train_sequences) // 64  # steps_per_epoch 계산
model.fit(
    dataset,
    epochs=5,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    callbacks=[checkpoint, early_stopping, tensorboard, csv_logger],
    verbose=2
)

Epoch 1/5
1/1 - 5s - 5s/step - loss: 3.8506 - val_loss: 3.8413
Epoch 2/5
1/1 - 0s - 110ms/step - loss: 3.8426 - val_loss: 3.8300
Epoch 3/5
1/1 - 0s - 128ms/step - loss: 3.8343 - val_loss: 3.8182
Epoch 4/5
1/1 - 0s - 142ms/step - loss: 3.8257 - val_loss: 3.8057
Epoch 5/5
1/1 - 0s - 72ms/step - loss: 3.8165 - val_loss: 3.7921


In [11]:
# 모델 저장
model.save('nickname_generator.keras')

In [ ]:
def generate_nickname_dl(seed_text, num_chars=10):
    # 딥러닝 모델 로드
    model = load_model("nickname_generator.h5")
    for _ in range(num_chars):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_length - 1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)  # 또는 model.predict()
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += output_word
    return seed_text

In [ ]:
print(generate_nickname_dl("별", 10))